# Проект по SQL - Анализ крупного сервиса для чтения книг по подписке

**Содержание**    
[1. Описание проекта](#description)    
[2. Загрузка данных](#loading)    
[3. Исследование данных](#research)    
[4. Задача 1](#task_1)    
[5. Задача 2](#task_2)    
[6. Задача 3](#task_3)    
[7. Задача 4](#task_4)    
[8. Задача 5](#task_5)    
[9. Выводы](#summary)    

## Описание проекта<a id="description"></a>    
Коронавирус застал мир врасплох, изменив привычный порядок вещей. В свободное время жители городов больше не выходят на улицу, не посещают кафе и торговые центры. Зато стало больше времени для книг. Это заметили стартаперы — и бросились создавать приложения для тех, кто любит читать.    
Ваша компания решила быть на волне и купила крупный сервис для чтения книг по подписке. Ваша первая задача как аналитика — проанализировать базу данных.
В ней — информация о книгах, издательствах, авторах, а также пользовательские обзоры книг. Эти данные помогут сформулировать ценностное предложение для нового продукта.  
   
**Задания**

- Посчитайте, сколько книг вышло после 1 января 2000 года;
- Для каждой книги посчитайте количество обзоров и среднюю оценку;
- Определите издательство, которое выпустило наибольшее число книг толще 50 страниц — так вы исключите из анализа брошюры;
- Определите автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками;
- Посчитайте среднее количество обзоров от пользователей, которые поставили больше 50 оценок.

**Как выполнить задание?**

- Опишите цели исследования;
- Исследуйте таблицы — выведите первые строки;
- Сделайте по одному SQL-запросу для решения каждого задания;
- Выведите результат каждого запроса в тетрадке;
- Опишите выводы по каждой из решённых задач.

### Описание данных

**Таблица `books`**   
   
Содержит данные о книгах:   
    
- `book_id` — идентификатор книги;  
- `author_id` — идентификатор автора;   
- `title` — название книги;   
- `num_pages` — количество страниц;   
- `publication_date` — дата публикации книги;   
- `publisher_id` — идентификатор издателя.   

**Таблица `authors`**   
   
Содержит данные об авторах:   
   
- `author_id` — идентификатор автора;   
- `author` — имя автора.   
   
**Таблица `publishers`**   
   
Содержит данные об издательствах:   

- `publisher_id` — идентификатор издательства;   
- `publisher` — название издательства;   
   
**Таблица `ratings`**   

Содержит данные о пользовательских оценках книг:   
   
- `rating_id` — идентификатор оценки;    
- `book_id` — идентификатор книги;    
- `username` — имя пользователя, оставившего оценку;   
- `rating` — оценка книги.  
   
**Таблица `reviews`**   
   
Содержит данные о пользовательских обзорах на книги:   
   
- `review_id` — идентификатор обзора;   
- `book_id` — идентификатор книги;   
- `username` — имя пользователя, написавшего обзор;   
- `text` — текст обзора.  


### Схема данных

In [1]:
from IPython.display import Image
display(Image(url='https://pictures.s3.yandex.net/resources/scheme_1589269096.png', width = 600))

---

## Загрузка данных<a id="loading"></a> 

In [2]:
%time
# импортируем библиотеки
import pandas as pd
from sqlalchemy import create_engine
# устанавливаем параметры
db_config = {'user': 'praktikum_student', # имя пользователя
'pwd': 'Sdf4$2;d-d30pp', # пароль
'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
'port': 6432, # порт подключения
'db': 'data-analyst-final-project-db'} # название базы данных
connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
 db_config['pwd'],
 db_config['host'],
 db_config['port'],
 db_config['db'])
# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'}) 

CPU times: user 4 µs, sys: 1 µs, total: 5 µs
Wall time: 10 µs


In [3]:
# Загружаем таблицу books
query = '''
            SELECT * FROM books
        '''

books = pd.io.sql.read_sql(query, con = engine)
books.name = 'books'

In [4]:
# Загружаем таблицу authors
query = '''
            SELECT * FROM authors
        '''

authors = pd.io.sql.read_sql(query, con = engine)
authors.name = 'authors'

In [5]:
# Загружаем таблицу publishers
query = '''
            SELECT * FROM publishers
        '''

publishers = pd.io.sql.read_sql(query, con = engine)
publishers.name = 'authors'

In [6]:
# Загружаем таблицу ratings
query = '''
            SELECT * FROM ratings
        '''

ratings = pd.io.sql.read_sql(query, con = engine)
ratings.name = 'authors'

In [7]:
# Загружаем таблицу reviews
query = '''
            SELECT * FROM reviews
        '''

reviews = pd.io.sql.read_sql(query, con = engine)
reviews.name = 'authors'

---

## Исследование данных<a id="research"></a>     
Для каждого из датасетов:   
* Выведем первую строку;
* Посмотрим описаение методом .info();   
* Проверим наличие дубликатов

In [8]:
print('Датасет books')
display(books.head(1))
print('-'*40)
books.info()
print('-'*40)
print('Дубликаты - ', books.duplicated().sum())

Датасет books


,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93


----------------------------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   book_id           1000 non-null   int64 
 1   author_id         1000 non-null   int64 
 2   title             1000 non-null   object
 3   num_pages         1000 non-null   int64 
 4   publication_date  1000 non-null   object
 5   publisher_id      1000 non-null   int64 
dtypes: int64(4), object(2)
memory usage: 47.0+ KB
----------------------------------------
Дубликаты -  0


In [9]:
print('Датасет authors')
display(authors.head(1))
print('-'*40)
authors.info()
print('-'*40)
print('Дубликаты - ', authors.duplicated().sum())

Датасет authors


,author_id,author
0,1,A.S. Byatt


----------------------------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 636 entries, 0 to 635
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   author_id  636 non-null    int64 
 1   author     636 non-null    object
dtypes: int64(1), object(1)
memory usage: 10.1+ KB
----------------------------------------
Дубликаты -  0


In [10]:
print('Датасет publishers')
display(publishers.head(1))
print('-'*40)
publishers.info()
print('-'*40)
print('Дубликаты - ', publishers.duplicated().sum())

Датасет publishers


,publisher_id,publisher
0,1,Ace


----------------------------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 340 entries, 0 to 339
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   publisher_id  340 non-null    int64 
 1   publisher     340 non-null    object
dtypes: int64(1), object(1)
memory usage: 5.4+ KB
----------------------------------------
Дубликаты -  0


In [11]:
print('Датасет ratings')
display(ratings.head(1))
print('-'*40)
ratings.info()
print('-'*40)
print('Дубликаты - ', ratings.duplicated().sum())

Датасет ratings


,rating_id,book_id,username,rating
0,1,1,ryanfranco,4


----------------------------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6456 entries, 0 to 6455
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   rating_id  6456 non-null   int64 
 1   book_id    6456 non-null   int64 
 2   username   6456 non-null   object
 3   rating     6456 non-null   int64 
dtypes: int64(3), object(1)
memory usage: 201.9+ KB
----------------------------------------
Дубликаты -  0


In [12]:
print('Датасет reviews')
display(reviews.head(1))
print('-'*40)
reviews.info()
print('-'*40)
print('Дубликаты - ', reviews.duplicated().sum())

Датасет reviews


,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...


----------------------------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2793 entries, 0 to 2792
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   review_id  2793 non-null   int64 
 1   book_id    2793 non-null   int64 
 2   username   2793 non-null   object
 3   text       2793 non-null   object
dtypes: int64(2), object(2)
memory usage: 87.4+ KB
----------------------------------------
Дубликаты -  0


**Вывод**  
Мы загрузили и изучили данные: пропусков и дубликатов в данных нет

---

## Задача 1 - Посчитайте, сколько книг вышло после 1 января 2000 года <a id="task_1"></a> 

In [13]:
query = '''
SELECT 
    COUNT(book_id) 
FROM 
    books
WHERE 
    publication_date >= '2000-01-01'
'''

number_books = pd.io.sql.read_sql(query, con = engine)
print('C 1 января 2000 года выпущено', number_books.loc[:, 'count'][0], 'книги')

C 1 января 2000 года выпущено 821 книги


---

## Задача 2 - Для каждой книги посчитайте количество обзоров и среднюю оценку<a id="task_2"></a> 

In [14]:
query = '''
SELECT 
    book.title,
    CASE
        WHEN review.reviews_num IS NULL THEN 0
        ELSE review.reviews_num
    END AS reviews_num,
    CASE
        WHEN rating.avg_rating IS NULL THEN 0
        ELSE rating.avg_rating
    END AS avg_rating
FROM
    books book
LEFT JOIN 
    (SELECT
        book_id,
        COUNT(review_id) AS reviews_num
    FROM 
        reviews
    GROUP BY 
        book_id
    ) AS review ON book.book_id = review.book_id
LEFT JOIN 
    (SELECT
        book_id,
        AVG(rating :: real) AS avg_rating
    FROM 
        ratings
    GROUP BY 
        book_id
    ) AS rating ON book.book_id = rating.book_id    
ORDER BY
    reviews_num DESC
LIMIT 5
'''

book_count_mean = pd.io.sql.read_sql(query, con = engine)
print('Число обзоров и средняя оценка книг')
book_count_max = book_count_mean.loc[:, 'title'][0]
display(book_count_mean)

Число обзоров и средняя оценка книг


,title,reviews_num,avg_rating
0,Twilight (Twilight #1),7,3.662500
1,The Hobbit or There and Back Again,6,4.125000
2,The Curious Incident of the Dog in the Night-Time,6,4.081081
3,Water for Elephants,6,3.977273
4,The Da Vinci Code (Robert Langdon #2),6,3.830508


In [15]:
print('Книга-лидер по числу обзоров -', book_count_max)

Книга-лидер по числу обзоров - Twilight (Twilight  #1)


---

## Задача 3 - Определите издательство, которое выпустило наибольшее число книг толще 50 страниц — так вы исключите из анализа брошюры<a id="task_3"></a> 

In [16]:
query = '''
SELECT 
    publisher.publisher,
    CASE
        WHEN book.books_num IS NULL THEN 0
        ELSE book.books_num
    END AS books_num
FROM
    publishers publisher
LEFT JOIN 
    (SELECT
        publisher_id,
        COUNT(book_id) AS books_num
    FROM 
        books
    WHERE
        num_pages >= 50
    GROUP BY 
        publisher_id
    ) AS book ON publisher.publisher_id = book.publisher_id
ORDER BY
    books_num DESC
LIMIT 1
'''
publisher_max = pd.io.sql.read_sql(query, con = engine)
publisher_max_name = publisher_max.loc[:, 'publisher'][0]
publisher_max_book = publisher_max.loc[:, 'books_num'][0]
print('Издательство, которое выпустило наибольшее число книг толще 50 страниц -',publisher_max_name, '-', publisher_max_book, 'книги')

Издательство, которое выпустило наибольшее число книг толще 50 страниц - Penguin Books - 42 книги


---

## Задача 4 - Определите автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками<a id="task_4"></a> 

In [17]:
query = '''
SELECT 
    author.author,
    AVG(avg_rating) AS avg_rating
FROM
    books book
INNER JOIN 
    (SELECT 
        book_id,
        COUNT(rating_id) AS ratings_num,
        AVG(rating) AS avg_rating
    FROM
        ratings
    GROUP BY
        book_id 
    HAVING 
        COUNT(rating_id) >= 50
    ) AS subq_b ON subq_b.book_id = book.book_id
INNER JOIN authors author ON author.author_id = book.author_id
GROUP BY
    author.author_id,
    author.author
ORDER BY
    avg_rating DESC
LIMIT 1
'''
best_raiting = pd.io.sql.read_sql(query, con = engine)
best_raiting_name = best_raiting.loc[:, 'author'][0]
best_raiting_avg = best_raiting.loc[:, 'avg_rating'][0].round(2)
print('Автор с самой высокой средней оценкой книг -', best_raiting_name, '. Рейтинг - ', best_raiting_avg)

Автор с самой высокой средней оценкой книг - J.K. Rowling/Mary GrandPré . Рейтинг -  4.28


---

## Задача 5 - Посчитайте среднее количество обзоров от пользователей, которые поставили больше 50 оценок<a id="task_5"></a> 

In [18]:
query = '''
SELECT
    AVG(reviews_num)
FROM
    (SELECT
        review.username,
        COUNT(*) AS reviews_num
    FROM
        reviews review
    INNER JOIN 
        (SELECT 
            username
        FROM
            ratings
        GROUP BY
            username 
        HAVING 
            COUNT(rating_id) > 50
        ) AS users ON users.username = review.username
    GROUP BY
        review.username
    ) AS reviews
'''

average_number_reviews = pd.io.sql.read_sql(query, con = engine)
average  = average_number_reviews.loc[:, 'avg'][0].round(2)
print('Среднее количество обзоров от пользователей', average)

Среднее количество обзоров от пользователей 24.33


---

## Выводы<a id="summary"></a>    

In [19]:
print('1. Мы загрузили и изучили данные: пропусков и дубликатов в данных нет')
print('2. C 1 января 2000 года выпущено', number_books.loc[:, 'count'][0], 'книги')
print('3. Книга-лидер по числу обзоров -', book_count_max, '🩸💗')
print('4. Издательство, которое выпустило наибольшее число книг толще 50 страниц -',publisher_max_name, '-', publisher_max_book, 'книги')
print('5. Автор с самой высокой средней оценкой книг -', best_raiting_name, '. Её рейтинг - ', best_raiting_avg, '🏰🐍')
print('6. Среднее количество обзоров от пользователей', average)

1. Мы загрузили и изучили данные: пропусков и дубликатов в данных нет
2. C 1 января 2000 года выпущено 821 книги
3. Книга-лидер по числу обзоров - Twilight (Twilight  #1) 🩸💗
4. Издательство, которое выпустило наибольшее число книг толще 50 страниц - Penguin Books - 42 книги
5. Автор с самой высокой средней оценкой книг - J.K. Rowling/Mary GrandPré . Её рейтинг -  4.28 🏰🐍
6. Среднее количество обзоров от пользователей 24.33
